In [ ]:
from ete3 import Tree
import os
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd

from simulate import simulate_shapes
from bridge_sampling.helper_functions import get_flat_values_sim


: 

In [ ]:
def generate_circle_landmarks(n_landmarks, radius=1.0, center_x=0.0, center_y=0.0, noise=0.0):
    """
    Generate landmarks equally spaced around a circle.
    
    Args:
        n_landmarks: Number of landmarks to generate
        radius: Radius of the circle
        center_x: X-coordinate of circle center
        center_y: Y-coordinate of circle center
        noise: Standard deviation of random noise to add (0 for perfect circle)
        
    Returns:
        Array of shape (n_landmarks*2,) with alternating x,y coordinates
    """
    # Generate angles equally spaced around the circle
    angles = np.linspace(0, 2 * np.pi, n_landmarks, endpoint=False)
    
    # Calculate x and y coordinates
    x_coords = center_x + radius * np.cos(angles)
    y_coords = center_y + radius * np.sin(angles)
    
    # Add random noise if specified
    if noise > 0:
        x_coords += np.random.normal(0, noise, n_landmarks)
        y_coords += np.random.normal(0, noise, n_landmarks)
    
    # Interleave x and y coordinates
    landmarks = np.zeros(n_landmarks * 2)
    landmarks[0::2] = x_coords
    landmarks[1::2] = y_coords
    
    return landmarks

In [ ]:
circle = generate_circle_landmarks(40)
plt.plot(circle[0::2], circle[1::2], 'ro')

In [ ]:
# load phylogeny 
tree = Tree('../data/chazot_subtree_rounded.nw')
print(tree)
# load root shape for simulation 
root = np.genfromtxt('../data/hercules_forewing_n=20.csv', delimiter=',')

: 

In [ ]:
leafidx = []
inneridx = []
species = []
i = 0
for node in tree.traverse('levelorder'):
    if node.is_leaf():
        print(node.name)
        leafidx.append(i)
        species.append(node.name)
    else:
        inneridx.append(i)
    i+=1
print(leafidx)
print(inneridx)
print(species)

: 

In [ ]:
ds = np.random.randint(0, 2**32)
sigma = 0.9
alpha = 0.02
dt = 0.05
outputpath = 'seed={}_sigma={}_alpha={}_dt={}'.format(ds, sigma, alpha, dt)
os.makedirs(outputpath, exist_ok=True)
simtree = simulate_shapes(ds=ds, dt=dt, sigma=sigma, alpha=alpha, root=root, tree=tree, rb=0)
flat_true_tree = np.array(get_flat_values_sim(simtree)) 
flat_true_tree.shape
#

: 

In [ ]:
np.savetxt(f'{outputpath}/flat_true_tree.csv', flat_true_tree, delimiter=",")

: 

In [ ]:
# show leaves 
[plt.plot(flat_true_tree[idx][0::2], flat_true_tree[idx][1::2], '--o', markersize=3, label='Simulated data') for idx in leafidx]
plt.savefig(f'{outputpath}/simulated_leaves.pdf', bbox_inches='tight')

: 

In [ ]:
leaves = pd.DataFrame(flat_true_tree[leafidx, :], index=species)
#leaves.columns = [['X'+str(i), 'Y'+str(i)] for i in range(leaves.shape[1] // 2)]
leaves.columns = [f'X{i}' if j % 2 == 0 else f'Y{i}' 
                  for i in range(leaves.shape[1] // 2) for j in range(2)]
print(leaves)
#path = f'comparison/seed={ds}_sigma={sigma}_alpha={alpha}_dt={dt}'
#os.makedirs(path, exist_ok=True)
leaves.to_csv(f'{outputpath}/leaves.csv')
flat_true_tree_df = pd.DataFrame(flat_true_tree)
flat_true_tree_df.to_csv(f'{outputpath}/flat_true_tree.csv', index=False)

: 

: 